In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('Questions.csv')
df.columns.values

array(['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score',
       'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditDate', 'LastActivityDate', 'Title',
       'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount',
       'ClosedDate'], dtype=object)

In [3]:
df['Tags'][:5]

0                       <python><pandas>
1                               <pandas>
2                       <python><pandas>
3                <python><pandas><apply>
4    <python><tuples><pandas><intervals>
Name: Tags, dtype: object

### Fill in missing CommentCount, AnswerCount, and FavoriteCount fields with 0

In [4]:
df['CommentCount'].fillna(0, inplace=True)
df['AnswerCount'].fillna(0, inplace=True)
df['FavoriteCount'].fillna(0, inplace=True)

### Create a CanonicalTags column

The first question has tag "<python\><pandas\>". Some other question could have tag "<pandas\><python\>". These would look different to Pandas, but we know they are really the same.

CanonicalTags is a string with the tags in alphabetical order (and we'll also drop the '<' and '>').

    * The CanonicalTags for "<python><pandas>" is "pandas python"
    * The CanonicalTags for "<pandas><python>" is also "pandas python"
    * The CanonicalTags for "<python><tuples><pandas><intervals>" is "intervals pandas python tuples"

In [5]:
def canonical_tags(s):
    return ' '.join(sorted(s.lstrip('<').rstrip('>').split('><')))

df['CanonicalTags']=df['Tags'].map(canonical_tags)
df['CanonicalTags'][:5]

0                     pandas python
1                            pandas
2                     pandas python
3               apply pandas python
4    intervals pandas python tuples
Name: CanonicalTags, dtype: object

### For each CanonicalTags, find the maximum AnswerCount among all questions about that CanonicalTags.

Sort the CanonicalTags in decreasing order of maximum AnswerCount.

In [6]:
df.groupby('CanonicalTags')['AnswerCount'].max().sort_values(ascending=False)

CanonicalTags
enthought numpy pandas python scipy                       7.0
easy-install numpy pandas pip python                      6.0
date pandas python                                        5.0
pandas python                                             5.0
algorithm average numpy pandas python                     5.0
                                                         ... 
distutils installation osx pandas python-3.3              0.0
bash pandas python root terminal                          0.0
inheritance pandas python series                          0.0
histogram matplotlib pandas                               0.0
hierarchical-data multi-index pandas python regression    0.0
Name: AnswerCount, Length: 337, dtype: float64

### Find popular CanonicalTags

Find the CanonicalTags for which there are 5 or more questions with those CanonicalTags.
Sort these CanonicalTags in decreasing order of mean CommentCount for questions with these CanonicalTags.

For example, for CanonicalTags="numpy pandas python", there are 31 questions, with a mean CommentCount of 1.42

In [7]:
df2 = df.groupby('CanonicalTags')['CommentCount'].agg(['count', 'mean'])
df2[df2['count']>=5].sort_values(by='mean', ascending=False)

,count,mean
CanonicalTags,,
numpy pandas python scipy,5,3.600000
data-analysis pandas python,5,2.000000
matplotlib pandas,7,1.857143
ipython pandas python,6,1.666667
numpy pandas python,31,1.419355
pandas python,351,1.005698
pandas,73,0.808219
pandas plot python,5,0.800000
datetime pandas python,8,0.750000
